<a href="https://colab.research.google.com/github/bundickm/Lambda-Treasure-Hunt/blob/master/Treasure_Hunt_Scratch_Pad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
from random import choice
from time import sleep
import json

api_key = '056a437a5b897e5082708ef06ce43b2d38e96ec1'
url_base = 'https://lambda-treasure-hunt.herokuapp.com/api/adv/'
headers = {'Content-Type': 'application/json',
           'Authorization': 'Token ' + api_key} 

In [0]:
def get_init():
  response = requests.get(url_base + 'init', headers=headers)
  status_code = response.status_code

  if status_code is not 200:
    return ('Unsuccessful Connection, Status Code:', status_code)
  return response.json()

In [0]:
def get_status():
  response = requests.post(url_base + 'status', headers=headers)
  status_code = response.status_code

  if status_code is not 200:
    return ('Unsuccessful Connection, Status Code:', status_code)
  return response.json()

In [0]:
def examine(target_json):
  response = requests.post(url_base + 'examine', headers=headers, json=target_json)
  status_code = response.status_code

  if status_code is not 200:
    return ('Unsuccessful Connection, Status Code:', status_code)
  return response.json()

# Examples
# examine({'name':'<NAME OF PLAYER OR ITEM>'})
# examine({'name':'tiny treasure'})

In [0]:
def make_move(move_json):
  response = requests.post(url_base + 'move', headers=headers, json=move_json)
  status_code = response.status_code

  if status_code is not 200:
    return ('Unsuccessful Connection, Status Code:', status_code)
  return response.json()

# Examples
# make_move({'direction':'s'})
# make_move('{"direction":"s", "next_room_id": "0"}')

In [0]:
def get_item(item_json):
  response = requests.post(url_base + 'take', headers=headers, json=item_json)
  status_code = response.status_code

  if status_code is not 200:
    return ('Unsuccessful Connection, Status Code:', status_code)
  return response.json()

# Example
# get_item({'name':'treasure'})

In [0]:
def drop_item(item_json):
  response = requests.post(url_base + 'drop', headers=headers, json=item_json)
  status_code = response.status_code

  if status_code is not 200:
    return ('Unsuccessful Connection, Status Code:', status_code)
  return response.json()

In [0]:
def change_name(name):
  response = requests.post(url_base + 'change_name', headers=headers, json={'name': name, 'confirm': 'aye'})
  status_code = response.status_code

  if status_code is not 200:
    return ('Unsuccessful Connection, Status Code:', status_code)
  return response.json()

In [0]:
def shop_check(data):
  if data['title'] == 'Shop':
      inventory = get_status()['inventory']
      for item in inventory:
        data = sell_item({'name': item, 'confirm': 'yes'})
        print('Sold Item')
        sleep(data['cooldown'] + 1)
      print('All items sold')
  return data

In [0]:
def sell_item(item_json):
  room_title = get_init()['title']
  if room_title != 'Shop':
    return 'You must be in the shop to sell items'
  sleep(1)
  response = requests.post(url_base + 'sell', headers=headers, json=item_json)
  status_code = response.status_code

  if status_code is not 200:
    return ('Unsuccessful Connection, Status Code:', status_code)
  return response.json()

# Examples
# sell_item({'name':'tiny treasure'}) # Will return sell price message but not complete transaction
# sell_item({'name':'tiny treasure', 'confirm':'yes'})

In [0]:
def extract_room_info(data):
  room = {}
  room_descriptors = ['title', 'coordinates', 'description',
                      'terrain', 'elevation', 'exits', 'items']
  for value in room_descriptors:
    room[value] = data[value]
  
  return room

# Example
# extract_room_info(response.json())

In [0]:
def mine(proof):
    response = requests.post(url_base[:-4] + 'bc/mine', headers=headers, json={'proof': proof})
    status_code = response.status_code

    if status_code is not 200:
      return ('Unsuccessful Connection, Status Code:', status_code)
    return response.json()

In [0]:
def get_cooldown():
  return get_status()['cooldown']

In [0]:
def make_blank_map(room_count = 500):
  rooms = {}
  for i in range(room_count):
    rooms[i] = {"n": None, "s": None, "e": None, "w": None,
                "title": None, "terrain": None, "elevation": None,
                "coordinates": None, "description": None}
  
  return rooms

In [0]:
def find_unvisited_directions(exits, current_room):
  unvisited = []
  for direction in exits:
    if current_room[direction] is None:
      unvisited.append(direction)
  return unvisited

In [0]:
def json_to_file(data, file_name):
  print('Saving File...')
  with open(file_name, 'w') as outfile:
    json.dump(data, outfile)
  print('Save Complete')

In [0]:
def file_to_json(filename):
    with open(filename) as json_file:
        file = json.load(json_file)
    return file

In [0]:
def pickup_item_check(data):
  error = None
  while (data['items'] != []) and (not error):
    data = get_item({'name': data['items'][0]})
    error = data['errors']
    if not error:
      print('Picked up item')
    sleep(data['cooldown']+1)
  return data

In [0]:
def drop_item_check(data):
  status = get_status()
  if status['encumbrance'] > status['strength']:
    data = drop_item({'name':'treasure'})
    print('Dropped Item')
    sleep(data['cooldown']+1)
  else:
    sleep(status['cooldown']+1)

  return data

In [0]:
def item_check(data):
  data = pickup_item_check(data)
  data = drop_item_check(data)
  return data

In [0]:
map = file_to_json('map.txt')
reverse = {'n':'s', 'e':'w', 's':'n', 'w':'e'}
backtrack = []
rooms_visited = 1
data = get_init()
rooms = make_blank_map()
sleep(data['cooldown']+1)

while rooms_visited < 500:
  current_room_id = data['room_id']
  room_data = extract_room_info(data)
  
  for key in room_data:
    rooms[current_room_id][key] = room_data[key]
  
  data = shop_check(data)
  data = item_check(data)

  exits = data['exits']
  valid_moves = find_unvisited_directions(exits, rooms[current_room_id])

  if valid_moves != []:
    move = choice(valid_moves)
    data = make_move({'direction': move, 
                      "next_room_id": str(map[str(current_room_id)][move])})
    backtrack.append(reverse[move])
    rooms[current_room_id][move] = data['room_id']
    rooms[data['room_id']][reverse[move]] = current_room_id
    rooms_visited += 1
    print(rooms_visited)
  else:
    move = backtrack.pop(-1)
    data = make_move({'direction': move, 
                      'next_room_id': str(map[str(current_room_id)][move])})

  sleep(data['cooldown']+1)

In [0]:
json_to_file(rooms, 'map.txt')

In [0]:
map = file_to_json('full_map.txt')

In [0]:
def find_path(start, end, map):
  best = None
  if start == end:
    return []
  while (best is None) or (len(best) > 250):
    path = []
    current_room = start
    while current_room != end:
      move = map[current_room][choice(['n','s','e','w'])]
      if (move is not None):
        if (path == []) or (move != path[-1]):
          path.append(current_room)
          current_room = str(move)
        else:
          current_room = path.pop()
    path.append(move)
    if (best is None) or (len(path) < len(best)):
      best = path
  return best
# path = find_path('431', '0', map)

In [0]:
room_names = {room['title'] for room in map.values()}
room_names

{'A Dark Cave',
 'A brightly lit room',
 'A misty room',
 "Glasowyn's Grave",
 "Linh's Shrine",
 'Mt. Holloway',
 None,
 "Pirate Ry's",
 'Shop',
 'The Peak of Mt. Holloway',
 'The Transmogriphier',
 'Warp Shrine',
 'Wishing Well'}

In [0]:
def trim_path(current_room, path):
  cut = 0
  for i in range(len(path)):
    if path[i] == current_room:
      cut = i
  return path[cut+1:]

In [0]:
def get_move(target_room, data, map):
  current_room = str(data['room_id'])
  exits = data['exits']

  for direction in exits:
    if map[current_room][direction] == int(target_room):
      return direction

In [0]:
KEY_ROOMS = {"The Transmogriphier": 495,
             "The Peak of Mt. Holloway": 22,
             "Pirate Ry's": 467,
             "A brightly lit room": 0,
             "A Dark Cave": 488,
             "Linh's Shrine": 461,
             "Glasowyn's Grave": 499,
             "Wishing Well": 55,
             "Shop": 1}

def move_to_location(current_room, destination, data, map):
  current_room = str(current_room)
  destination = str(destination)

  path = find_path(current_room, destination, map)
  while len(path) > 1:
    print('path length:', len(path))
    path = trim_path(current_room, path)
    print('trimmed length:',len(path))
    print(path[:5])
    move = str(get_move(path[0], data, map))
    data = make_move({'direction': move,
                      'next_room_id': str(map[current_room][move])})

    current_room = str(data['room_id'])
    sleep(data['cooldown'] + 1)

  return data

In [0]:
data = get_init()
data

{'cooldown': 1.0,
 'coordinates': '(66,55)',
 'description': 'You are standing on grass and surrounded by a dense mist. You can barely make out the exits in any direction.',
 'elevation': 0,
 'errors': [],
 'exits': ['e', 'w'],
 'items': ['shiny treasure'],
 'messages': [],
 'players': [],
 'room_id': 154,
 'terrain': 'NORMAL',
 'title': 'A misty room'}

In [0]:
move_to_location(data['room_id'], 154, data, map)

{'cooldown': 1.0,
 'coordinates': '(66,55)',
 'description': 'You are standing on grass and surrounded by a dense mist. You can barely make out the exits in any direction.',
 'elevation': 0,
 'errors': [],
 'exits': ['e', 'w'],
 'items': ['shiny treasure'],
 'messages': [],
 'players': [],
 'room_id': 154,
 'terrain': 'NORMAL',
 'title': 'A misty room'}

In [0]:
def list_to_file(data, filename):
  with open(filename, 'w') as filehandle:
      for line in data:
          filehandle.write('%s\n' % line)

In [0]:
def list_to_file(data, filename):
    with open(filename, 'w') as filehandle:
        for line in data:
            filehandle.write('%s\n' % line)


def get_binary_from_well():
    binary = examine({'name':'Wishing Well'})['description']
    binary = binary.split('\n')[2:]

    list_to_file(binary, 'binary.txt')

In [0]:
import sys


IM = 5  # R5 = interrupt mask (IM)
IS = 6  # R6 = interrupt status (IS)
SP = 7  # R7 = stack pointer (SP)

FL_EQ = 0b001
FL_GT = 0b010
FL_LT = 0b100

class CPU:
    """Main CPU class."""

    def __init__(self):
        self.registers = [0b0] * 8  # Registers R0-R7
        self.ram = [0] * 256

        self.message = []
        self.halted = False

        self.pc = 0 # Program Counter, location of current instruction executing
        self.fl = 0 # Flags
        self.ir = None # Instruction Register, current instruction executing

        self.ram = [0b0] * 0xFF
        self.registers[SP] = 0xf4

        self.OPCODES = {
            0b01000111: 'PRN',
            0b00000001: 'HLT',
            0b10000010: 'LDI',
            0b10100000: 'ADD',
            0b10100010: 'MUL',
            0b01000101: 'PUSH',
            0b01000110: 'POP',
            0b10000100: 'STOR',
            0b01010000: 'CALL',
            0b00010001: 'RET',
            0b10100111: 'CMP',
            0b01010100: 'JMP',
            0b01010101: 'JEQ',
            0b01010110: 'JNE',
            0b01001000: 'PRA',
        }

    def load(self, filename):
        """Load a program into memory."""
        address = 0

        try:
            # Read in the program
            with open(filename, 'r') as file:
                lines = (line for line in file.readlines() 
                        if not (line[0]=='#' or line[0]=='\n'))
                program = [int(line.split('#')[0].strip(), 2) 
                          for line in lines]

            for instruction in program:
                self.ram[address] = instruction
                address += 1
        except FileNotFoundError as e:
            print(e)
            sys.exit()

    def alu(self, op, reg_a, reg_b):
        """ALU operations."""
        if op == 'ADD':
            self.registers[reg_a] += self.registers[reg_b]
        elif op == 'SUB':
            self.registers[reg_a] -= self.registers[reg_b]
        elif op == 'MUL':
            self.registers[reg_a] *= self.registers[reg_b]
        elif op == 'DIV':
            self.registers[reg_a] //= self.registers[reg_b]
        elif op == 'CMP':
            self.fl &= 0x11111000  # Clear flags
            if self.registers[reg_a] < self.registers[reg_b]:
                self.fl |= FL_LT
            elif self.registers[reg_a] > self.registers[reg_b]:
                self.fl |= FL_GT
            else:
                self.fl |= FL_EQ
        else:
            raise Exception("Unsupported ALU operation")

    def trace(self):
        """
        Handy function to print out the CPU state. You might want to call this
        from run() if you need help debugging.
        """
        print(f"TRACE: %02X | %02X %02X %02X |" % (
            self.pc,
            self.fl,
            self.ie,
            self.ram_read(self.pc),
            self.ram_read(self.pc + 1),
            self.ram_read(self.pc + 2)
        ), end='')

        for i in range(8):
            print(" %02X" % self.reg[i], end='')

        print()

    def ram_write(self, mdr, mar):
        self.ram[mar] = mdr

    def ram_read(self, mar):
        return self.ram[mar]

    def op_prn(self, r1):
        val = self.registers[r1]
        print(f'value: {val}')
        self.pc += 2

    def op_ldi(self, r1, r2):
        self.registers[r1] = r2
        self.pc += 3

    def op_alu(self, op, r1, r2):
        self.alu(op, r1, r2)
        self.pc += 3

    def op_push(self, r1):
        val = self.registers[r1]
        self.registers[SP] -= 1
        self.ram_write(val, self.registers[SP])
        self.pc += 2

    def op_pop(self, r1):
        val = self.ram_read(self.registers[SP])
        self.registers[r1] = val
        self.registers[SP] += 1
        self.pc += 2

    def op_call(self, r1):
        self.registers[SP] -= 1
        self.ram_write(self.pc + 2, self.registers[SP])
        self.pc = self.registers[r1]

    def op_ret(self):
        val = self.ram_read(self.registers[SP])
        self.pc = val
        self.registers[SP] += 1

    def op_jmp(self, r1):
        self.pc = self.registers[r1]

    def op_jeq(self, r1):
        if self.fl & FL_EQ:
            self.pc = self.registers[r1]
        else:
            self.pc += 2

    def op_jne(self, r1):
        if not self.fl & FL_EQ:
            self.pc = self.registers[r1]
        else:
            self.pc += 2

    def op_pra(self, r1):
        self.message.append(chr(self.registers[r1]))
        self.pc += 2

    def run(self):
        while not self.halted:
            self.ir = self.ram[self.pc]
            op = self.OPCODES[self.ir]
            r1 = self.ram_read(self.pc + 1)
            r2 = self.ram_read(self.pc + 2)

            if op == 'LDI': # Save to Register
                self.op_ldi(r1, r2)
            elif op == 'PRN': # Print
                self.op_prn(r1)
            elif op in ['ADD', 'MUL', 'SUB', 'DIV', 'CMP']: # Math
                self.op_alu(op, r1, r2)
            elif op == 'PUSH': # Write to Stack
                self.op_push(r1)
            elif op == 'POP': # Pull from Stack
                self.op_pop(r1)
            elif op == 'CALL': # Call subroutine
                self.op_call(r1)
            elif op == 'RET':   # Return from subroutine
                self.op_ret()
            elif op == 'JMP': # Jump
                self.op_jmp(r1)
            elif op == 'JEQ': # Jump if equal
                self.op_jeq(r1)
            elif op == 'JNE': # Jump if not equal
                self.op_jne(r1)
            elif op == 'PRA':
                self.op_pra(r1)
            elif op == 'HLT': # Exit
                self.halted = True
                return self.message
            else:
                print(f'Unknown Operation: {op}')
                self.halted = True

In [0]:
def translate_binary(filename):
    cpu = CPU()
    cpu.load(filename)
    return ''.join(cpu.run())

translate_binary('binary.txt')

'Mine your coin in room 154'

In [0]:
def mine(proof):
    response = requests.post(url_base[:-4] + 'bc/mine', headers=headers, json={"proof": proof})
    status_code = response.status_code

    if status_code is not 200:
      return ('Unsuccessful Connection, Status Code:', status_code, response.json()['errors'])
    return response.json()

In [0]:
mine(999999999)

('Unsuccessful Connection, Status Code:', 400, ['Invalid proof: +30s CD'])

In [0]:
def get_last_proof():
    response = requests.get(url_base[:-4] + 'bc/last_proof', headers=headers)
    status_code = response.status_code

    if status_code is not 200:
      return ('Unsuccessful Connection, Status Code:', status_code)
    return response.json()

In [0]:
data = get_last_proof()
data

{'cooldown': 1.0,
 'difficulty': 6,
 'errors': [],
 'messages': [],
 'proof': 9690227}

In [0]:
proof = ''
while proof == '':
  proof = proof_of_work(data['proof'], data['difficulty'])
proof

Searching for next proof
Proof found: 785319023 in 17.482626156999913


785319023

In [0]:
valid_proof(data['proof'], proof, data['difficulty'])

True

In [0]:
mine(proof)

{'cooldown': 1.0,
 'coordinates': '(66,55)',
 'description': 'You are standing on grass and surrounded by a dense mist. You can barely make out the exits in any direction.',
 'elevation': 0,
 'errors': [],
 'exits': ['e', 'w'],
 'items': ['shiny treasure'],
 'messages': [],
 'players': [],
 'room_id': 154,
 'terrain': 'NORMAL',
 'title': 'A misty room'}

In [0]:
message = []
response = {'cooldown': -1}
while message == []:
  print('sleep:', response['cooldown']+1)
  sleep(response['cooldown']+1)
  data = get_last_proof()

  proof = proof_of_work(data['proof'], data['difficulty'])
  response = mine(proof)
  print(response)
  message = response['messages']

sleep: 0
Searching for next proof
Proof found: 453871009067 in 25.621530722000898
{'name': 'Michael Bundick', 'description': 'Michael Bundick looks like an ordinary person.', 'cooldown': 115.0, 'errors': ['There is no coin here: +100s'], 'messages': []}
sleep: 116.0


In [0]:
get_balance()

{'cooldown': 1.0,
 'errors': [],
 'messages': ['You have a balance of 1.0 Lambda Coins']}

In [0]:
get_init()

{'cooldown': 1.0,
 'coordinates': '(66,55)',
 'description': 'You are standing on grass and surrounded by a dense mist. You can barely make out the exits in any direction.',
 'elevation': 0,
 'errors': [],
 'exits': ['e', 'w'],
 'items': ['shiny treasure'],
 'messages': [],
 'players': [],
 'room_id': 154,
 'terrain': 'NORMAL',
 'title': 'A misty room'}

In [0]:
def get_balance():
  response = requests.get(url_base[:-4] + 'bc/get_balance', headers=headers)
  status_code = response.status_code

  if status_code is not 200:
    return ('Unsuccessful Connection, Status Code:', status_code)
  return response.json()

In [0]:
get_balance()

{'cooldown': 1.0,
 'errors': [],
 'messages': ['You have a balance of 1.0 Lambda Coins']}

In [0]:
def treasure_hunt():
  map = file_to_json('full_map.txt')
  reverse = {'n': 's', 'e': 'w', 's': 'n', 'w': 'e'}
  backtrack = []
  data = get_init()
  sleep(data['cooldown']+1)
  rooms = make_blank_map()
  inventory = get_status()['inventory']

  while len(inventory) < 7:
      current_room_id = data['room_id']
      room_data = extract_room_info(data)

      data = item_check(data)
      exits = data['exits']
      valid_moves = find_unvisited_directions(exits, rooms[current_room_id])

      if valid_moves != []:
          move = choice(valid_moves)
          data = make_move({'direction': move,
                            "next_room_id": str(map[str(current_room_id)][move])})
          backtrack.append(reverse[move])
          rooms[current_room_id][move] = data['room_id']
          rooms[data['room_id']][reverse[move]] = current_room_id
      else:
          move = backtrack.pop(-1)
          data = make_move({'direction': move,
                            'next_room_id': str(map[str(current_room_id)][move])})

      sleep(data['cooldown']+1)
      inventory = get_status()['inventory']
  return data

In [0]:
while get_status()['gold'] < 1000:
  data = treasure_hunt()
  data = move_to_location(data['room_id'], KEY_ROOMS['Shop'], data, map)
  data = shop_check(data)

data = move_to_location(data['room_id'], KEY_ROOMS["Pirate Ry's"], data, map)
# data = change_name('YOUR NAME HERE')
data = move_to_location(data['room_id'], KEY_ROOMS['Wishing Well'], data, map)
get_binary_from_well()
mine_location = int(translate_binary('binary.txt')[-3:])
data = move_to_location(data['room_id'], mine_location, data, map)

In [0]:
def proof_of_work(last_proof, difficulty):
    start = timer()

    print("Searching for next proof")
    proof = 0
    attempts = 0
    while valid_proof(last_proof, proof, difficulty) is False:
        proof = random.randint(0, 999999999999)
        attempts += 1

    print("Proof found: " + str(proof) + " in " + str(timer() - start))
    return proof


def valid_proof(last_proof, proof, difficulty):
    guess = f'{last_proof}{proof}'.encode()
    guess_hash = hashlib.sha256(guess).hexdigest()

    if difficulty is not None:
        leading_zeros = "0" * difficulty
    else:
        leading_zeros = "0" * 6
    
    return guess_hash[0:difficulty] == leading_zeros